In [42]:
import pandas as pd
import datetime

treshold = 10
fee = 1.5
delta = 0.95

trading_days_in_year = 252

start_date = datetime.date(year=2006,month=6,day=13)
end_date = datetime.date(year=2018,month=12,day=22)

gold = pd.read_csv('data/gold_daily.csv', index_col='Date')
gold_cot = pd.read_csv('data/gold_cot.csv', index_col='Date')

corn = pd.read_csv('data/corn_daily.csv', index_col='Date')
corn_cot = pd.read_csv('data/corn_cot.csv', index_col='Date')

coffee = pd.read_csv('data/coffee_daily.csv', index_col='Date')
coffee_cot = pd.read_csv('data/coffee_cot.csv', index_col='Date')

In [43]:
gold.index = pd.DatetimeIndex(data=gold.index)
gold_cot.index = pd.DatetimeIndex(data=gold_cot.index)

corn.index = pd.DatetimeIndex(data=corn.index)
corn_cot.index = pd.DatetimeIndex(data=corn_cot.index)

coffee.index = pd.DatetimeIndex(data=coffee.index)
coffee_cot.index = pd.DatetimeIndex(data=coffee_cot.index)

In [44]:
gold = gold.sort_index(kind='mergesort')
gold_cot = gold_cot.sort_index(kind='mergesort')
corn = corn.sort_index(kind='mergesort')
corn_cot = corn_cot.sort_index(kind='mergesort')
coffee = coffee.sort_index(kind='mergesort')
coffee_cot = coffee_cot.sort_index(kind='mergesort')

In [45]:
gold = gold.loc[start_date:end_date]
corn = corn.loc[start_date:end_date]
coffee = coffee.loc[start_date:end_date]

In [46]:
gold_cot.head(10)

,Open Interest,Producer/Merchant/Processor/User Longs,Producer/Merchant/Processor/User Shorts,Swap Dealer Longs,Swap Dealer Shorts,Swap Dealer Spreads,Money Manager Longs,Money Manager Shorts,Money Manager Spreads,Other Reportable Longs,Other Reportable Shorts,Other Reportable Spreads,Total Reportable Longs,Total Reportable Shorts,Non Reportable Longs,Non Reportable Shorts
Date,,,,,,,,,,,,,,,,
2006-06-13,382474.0,42587.0,128892.0,19370.0,66611.0,22998.0,103211.0,30916.0,46704.0,33230.0,10656.0,58412.0,326513.0,365190.0,55961.0,17284.0
2006-06-20,390281.0,47440.0,126992.0,22404.0,64682.0,25806.0,94632.0,30963.0,48730.0,32947.0,11142.0,64458.0,336417.0,372774.0,53864.0,17507.0
2006-06-27,384321.0,48449.0,121890.0,22425.0,62640.0,25151.0,91042.0,32287.0,49614.0,31307.0,11386.0,59762.0,327749.0,362729.0,56572.0,21592.0
2006-07-03,392868.0,48087.0,122568.0,22720.0,69714.0,26352.0,92545.0,30452.0,50821.0,32874.0,8677.0,63514.0,336914.0,372098.0,55955.0,20771.0
2006-07-11,427008.0,54509.0,130618.0,21840.0,79217.0,28805.0,104213.0,27624.0,54922.0,33165.0,11194.0,70944.0,368398.0,403323.0,58611.0,23685.0
2006-07-18,443519.0,49863.0,131166.0,21681.0,83183.0,33352.0,107413.0,24296.0,62395.0,31760.0,9442.0,74272.0,380737.0,418105.0,62782.0,25414.0
2006-07-25,432413.0,52859.0,133290.0,22956.0,85604.0,28832.0,99087.0,22386.0,64810.0,29740.0,6244.0,73980.0,372265.0,415146.0,60148.0,17266.0
2006-08-01,394609.0,45161.0,135988.0,21250.0,73244.0,23643.0,99330.0,24391.0,54618.0,32195.0,4758.0,57618.0,333815.0,374260.0,60795.0,20350.0
2006-08-08,402899.0,46203.0,142240.0,20703.0,72061.0,24259.0,104600.0,24271.0,57785.0,33123.0,6489.0,57914.0,344587.0,385020.0,58312.0,17880.0


In [61]:
def percentage_from_max(sr):
    mx = sr.max()
    mn = sr.min()
    return (sr[-1]-mn)/(mx-mn)

def max_positions_percentage(df):
    df['Prod_net_position'] = df['Producer/Merchant/Processor/User Longs'] - df['Producer/Merchant/Processor/User Shorts']
    df['Prod_net_position_perc'] = df.rolling('730d')['Prod_net_position'].apply(percentage_from_max, raw=False)
    df['Money_manager_net_position'] = df['Money Manager Longs'] - df['Money Manager Shorts']
    df['Money_manager_net_position_perc'] = df.rolling('730d')['Money_manager_net_position'].apply(percentage_from_max, raw=False)
    return df

In [58]:
gold_cot = max_positions_percentage(gold_cot)
corn_cot = max_positions_percentage(corn_cot)
coffee_cot = max_positions_percentage(coffee_cot)

In [67]:
# gold_cot[(gold_cot['Prod_net_position_perc'] == 1.0) | (gold_cot['Money_manager_net_position_perc'] == 1.0)][['Money_manager_net_position_perc', 'Prod_net_position_perc']]

In [68]:
def create_labels(df):
    global fee, treshold, delta
    def calculate_target(row):
        volatility = row*delta
        if abs(volatility) > fee + treshold:
            if volatility > 0:
                return 1
            else:
                return (-1)
        else:
            return 0

    df['target'] = df['Settle'] - df['Open']
    df['target'] = df['target'].map(calculate_target)
    df.target = df.target.shift(-1)
    return df

In [69]:
gold = create_labels(gold)
tot, trade = gold.shape[0], gold[gold.target != 0].shape[0]
print(tot, trade, float(trade/tot))

3168 755 0.23832070707070707


In [70]:
corn = create_labels(corn)
tot, trade = corn.shape[0], corn[corn.target != 0].shape[0]
print(tot, trade, float(trade/tot))

3159 380 0.12029123140234252


In [71]:
treshold = 3
coffee = create_labels(coffee)
tot, trade = coffee.shape[0], coffee[coffee.target != 0].shape[0]
print(tot, trade, float(trade/tot))

3149 320 0.10161956176563988


In [72]:
coffee[coffee.target != 0].head(10)

,Open,High,Low,Settle,Volume,Prev. Day Open Interest,target
Date,,,,,,,
2007-05-31,111.05,112.25,110.10,111.90,10816.0,80301.0,1.0
2007-08-15,118.90,119.85,117.30,119.20,11777.0,35209.0,-1.0
2007-09-14,119.50,121.50,119.10,120.90,9543.0,102889.0,1.0
2007-09-28,129.90,130.40,127.50,128.65,10588.0,105013.0,1.0
2007-10-12,137.80,140.80,136.20,139.30,12363.0,104569.0,-1.0
2008-02-15,155.55,160.15,151.90,152.40,26985.0,114078.0,1.0
2008-03-05,163.70,165.50,161.55,163.90,13236.0,129113.0,-1.0
2008-03-13,155.20,159.10,154.50,157.85,11078.0,114127.0,-1.0
2008-03-14,159.00,159.90,148.30,152.20,22142.0,113367.0,-1.0


In [178]:
# from itertools import count
import numpy as np


def create_vectors(df):
    global trading_days_in_year
    def form_vector(sr, i, start):
        return sr[i]
    
    ans_df = pd.DataFrame([])
    for i in range(df.shape[0]):
        if i < 2*trading_days_in_year:
            continue
        mtrx = df.iloc[i-2*trading_days_in_year:i, 0:(df.shape[1]-1)].values
        size = np.prod(mtrx.shape)
        mtrx = list(mtrx.reshape(size))
        # print(mtrx[:10])
        idx = df.index.values[i]
        hdf = pd.DataFrame.from_dict(data={idx: mtrx}, orient='index')
        # print(hdf)
        ans_df.append(hdf)    
    return ans_df

In [ ]:
gold_data = create_vectors(gold)

In [ ]:
gold_data.head(10)

In [93]:
gold_data.head(10)

,Open,High,Low,Settle,Volume,Prev. Day Open Interest
Date,,,,,,
2006-06-13,590.5,595.0,565.5,566.8,93899.0,192616.0
2006-06-14,570.0,575.5,565.4,566.5,68729.0,192917.0
2006-06-15,573.5,579.5,569.5,570.3,52628.0,193887.0
2006-06-16,581.2,582.5,570.5,581.7,43947.0,189585.0
2006-06-19,572.8,578.4,571.0,572.4,27362.0,189168.0
2006-06-20,572.0,581.0,567.8,580.5,39962.0,187696.0
2006-06-21,577.0,591.5,573.5,591.0,43918.0,187072.0
2006-06-22,588.0,593.5,584.5,585.4,42916.0,187027.0
2006-06-23,574.5,589.0,574.5,588.0,33705.0,181290.0


In [28]:
coffee.head(10).values

array([[   97.95,    98.8 ,    97.9 ,    97.95, 17135.  , 58940.  ,
            0.  ],
       [   98.45,    99.4 ,    97.6 ,    98.45, 14428.  , 67177.  ,
            0.  ],
       [   98.95,    99.8 ,    98.5 ,    98.95, 12157.  , 70486.  ,
            0.  ],
       [   97.7 ,    99.6 ,    96.9 ,    97.7 , 13318.  , 72546.  ,
            0.  ],
       [   97.25,    97.9 ,    97.  ,    97.25, 11340.  , 75645.  ,
            0.  ],
       [   97.45,    99.1 ,    97.3 ,    97.45, 14405.  , 77949.  ,
            0.  ],
       [   96.7 ,    97.85,    96.6 ,    96.7 , 13144.  , 81345.  ,
            0.  ],
       [   97.1 ,    97.5 ,    96.3 ,    97.1 ,  8922.  , 83200.  ,
            0.  ],
       [   96.6 ,    97.2 ,    96.25,    96.6 ,  6959.  , 84049.  ,
            0.  ],
       [   95.95,    97.55,    95.85,    95.95,  8095.  , 84894.  ,
            0.  ]])

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components='mle', random_state=2234)
pca.fit(X)